In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [4]:
for root, dirs, file in os.walk('/Volumes/TBHD/Bioinformatics/ion_torrent_qiime2_methods_manuscript/Classified_Tax_Counts_QIIME2_CSV_Files/Cutprimers/Species'):
    for f in file:
        if "Silva" in f:
            df = pd.read_csv(os.path.join(root, f)).set_index('index')
            df.drop(columns=['even_stag', 'run', 'in_out', 'v_region'], inplace=True)
            # display(df)
            # display(df.T)

            new_cols = []
            cols = df.columns.to_list()
            for c in cols:
                pattern1 = re.compile(r';')
                col6 = re.split(pattern1, c)[-1]
                # print(col6)

                pattern2 = re.compile(r'._\d_+')
                col6_clean = re.split(pattern2, col6)[-1]

                new_cols.append(col6_clean)

            print(new_cols)
            df.columns = new_cols
            deduped = df.groupby(lambda x:x, axis=1).sum()
            display(deduped.T)

            # break

['uncultured bacterium', 'uncultured bacterium', '__', 'uncultured bacterium', 'uncultured bacterium', 'uncultured organism', '__', '__', '__', 'Listeria monocytogenes', 'Staphylococcus aureus', 'uncultured bacterium', '__', 'Enterococcus faecalis', 'Lactobacillus fermentum', 'uncultured bacterium', '__', 'Streptococcus agalactiae', 'Streptococcus mutans', 'uncultured bacterium', '__', 'uncultured bacterium', 'Rhodobacter sphaeroides', '__', '__', '__', '__', 'uncultured bacterium', '__', '__', '__', 'Pseudomonas aeruginosa', '__', '__']


index,even_or_12_V2,even_or_13_V2,even_or_19_V2,even_or_20_V2,even_or_21_V2,even_out_001_V2,even_out_006_V2,even_out_007_V2,even_out_008_V2,even_out_009_V2,...,stag_out_011_V2,stag_st_01_V2,stag_st_02_V2,stag_st_03_V2,stag_st_04_V2,stag_st_05_V2,stag_st_06_V2,stag_st_07_V2,stag_st_08_V2,stag_st_09_V2
Enterococcus faecalis,130.0,648.0,209.0,297.0,188.0,3.0,1377.0,1650.0,2260.0,2067.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lactobacillus fermentum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Listeria monocytogenes,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,...,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pseudomonas aeruginosa,620.0,2936.0,982.0,1284.0,945.0,1539.0,2785.0,3639.0,2712.0,2454.0,...,426.0,125.0,227.0,114.0,439.0,421.0,527.0,420.0,275.0,122.0
Rhodobacter sphaeroides,261.0,1204.0,359.0,645.0,207.0,1214.0,964.0,1455.0,1251.0,805.0,...,1771.0,635.0,1119.0,476.0,2225.0,1557.0,2202.0,1826.0,1270.0,534.0
Staphylococcus aureus,210.0,858.0,402.0,485.0,375.0,0.0,0.0,0.0,312.0,356.0,...,224.0,0.0,32.0,16.0,78.0,35.0,64.0,41.0,34.0,0.0
Streptococcus agalactiae,113.0,643.0,180.0,322.0,164.0,183.0,901.0,879.0,115.0,530.0,...,275.0,37.0,82.0,34.0,116.0,110.0,167.0,111.0,69.0,24.0
Streptococcus mutans,60.0,218.0,61.0,127.0,50.0,1415.0,50.0,17.0,32.0,136.0,...,1826.0,204.0,438.0,144.0,654.0,504.0,733.0,636.0,354.0,158.0
__,5166.0,24308.0,8013.0,11057.0,7482.0,12284.0,9360.0,17917.0,13905.0,19610.0,...,7735.0,15458.0,28905.0,12001.0,48134.0,39167.0,52399.0,38653.0,26838.0,11597.0
uncultured bacterium,2858.0,12691.0,4904.0,6415.0,5318.0,557.0,1688.0,2271.0,1816.0,2186.0,...,607.0,155.0,320.0,118.0,560.0,606.0,836.0,588.0,409.0,142.0


['uncultured bacterium', 'uncultured bacterium', 'uncultured bacterium', 'uncultured bacterium', 'uncultured organism', 'uncultured bacterium', '__', 'uncultured bacterium', 'uncultured bacterium', '__', '__', 'Streptomyces sp.', '__', 'Listeria monocytogenes', 'Staphylococcus aureus', 'uncultured bacterium', '__', 'Enterococcus faecalis', 'Lactobacillus fermentum', 'uncultured bacterium', '__', 'uncultured bacterium', 'Streptococcus agalactiae', 'Streptococcus mutans', 'uncultured bacterium', '__', 'uncultured bacterium', '__', 'uncultured bacterium', '__', '__', 'uncultured bacterium', '__', 'uncultured bacterium', '__', '__', 'Acinetobacter baumannii', '__', 'Pseudomonas aeruginosa', '__', '__', '__', '__']


index,even_or_12_V3,even_or_13_V3,even_or_19_V3,even_or_20_V3,even_or_21_V3,even_out_006_V3,even_out_007_V3,even_out_008_V3,even_out_009_V3,even_out_012_V3,...,stag_out_011_V3,stag_st_01_V3,stag_st_02_V3,stag_st_03_V3,stag_st_04_V3,stag_st_05_V3,stag_st_06_V3,stag_st_07_V3,stag_st_08_V3,stag_st_09_V3
Acinetobacter baumannii,0.0,0.0,10.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Enterococcus faecalis,2374.0,10200.0,4906.0,4953.0,4035.0,6791.0,5792.0,7965.0,5734.0,2647.0,...,9.0,10.0,25.0,7.0,31.0,33.0,56.0,63.0,16.0,10.0
Lactobacillus fermentum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Listeria monocytogenes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3022.0,...,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pseudomonas aeruginosa,3092.0,14984.0,6846.0,7509.0,5867.0,10703.0,9116.0,6960.0,5499.0,2539.0,...,1040.0,2002.0,4178.0,1727.0,6649.0,5663.0,8237.0,7024.0,3957.0,1554.0
Staphylococcus aureus,3385.0,15604.0,6669.0,7097.0,5482.0,1024.0,696.0,5105.0,1955.0,3120.0,...,1165.0,1999.0,3690.0,1569.0,6091.0,5253.0,7354.0,6539.0,3623.0,1470.0
Streptococcus agalactiae,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Streptococcus mutans,3699.0,17222.0,7279.0,7308.0,6390.0,2373.0,1736.0,12509.0,4822.0,3535.0,...,12412.0,22159.0,40414.0,17132.0,69464.0,59780.0,80409.0,73445.0,40593.0,17226.0
Streptomyces sp.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
__,23456.0,107886.0,45367.0,51673.0,37912.0,47901.0,44296.0,41996.0,49528.0,18485.0,...,24384.0,79610.0,139230.0,61302.0,247955.0,202553.0,272894.0,235695.0,133949.0,57685.0


['uncultured bacterium', '__', 'uncultured bacterium', 'uncultured bacterium', 'uncultured bacterium', 'uncultured organism', '__', 'uncultured organism', 'uncultured bacterium', '__', '__', 'Bacillus cereus', '__', 'Listeria monocytogenes', 'uncultured bacterium', '__', '__', 'uncultured Lactobacillaceae bacterium', 'uncultured bacterium', 'uncultured bacterium', '__', '__', 'Rhodobacter sphaeroides', '__', 'Neisseria meningitidis', '__', '__', 'Acinetobacter baumannii', '__', 'Pseudomonas aeruginosa', '__', '__']


...

index,even_or_12_V67,even_or_13_V67,even_or_19_V67,even_or_20_V67,even_or_21_V67,even_out_006_V67,even_out_007_V67,even_out_008_V67,even_out_009_V67,even_out_012_V67,...,stag_out_011_V67,stag_st_01_V67,stag_st_02_V67,stag_st_03_V67,stag_st_04_V67,stag_st_05_V67,stag_st_06_V67,stag_st_07_V67,stag_st_08_V67,stag_st_09_V67
Acinetobacter baumannii,33.0,144.0,68.0,75.0,34.0,65.0,60.0,14.0,20.0,45.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bacillus cereus,3317.0,17302.0,5816.0,7880.0,3702.0,5924.0,6091.0,6551.0,10978.0,789.0,...,293.0,2221.0,4383.0,1945.0,7982.0,5275.0,7579.0,7607.0,4018.0,1793.0
Bacillus subtilis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Escherichia coli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Listeria monocytogenes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2521.0,...,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pseudomonas aeruginosa,894.0,4149.0,1587.0,1865.0,1315.0,4012.0,3555.0,3087.0,2002.0,273.0,...,141.0,596.0,1018.0,477.0,1828.0,1473.0,1937.0,1584.0,1019.0,454.0
Staphylococcus aureus,1774.0,8395.0,3467.0,4424.0,2776.0,324.0,244.0,1681.0,746.0,789.0,...,253.0,1077.0,2385.0,1003.0,3929.0,3076.0,4430.0,3933.0,2196.0,816.0
Streptococcus mutans,1819.0,7899.0,3251.0,4114.0,2662.0,587.0,382.0,3325.0,1682.0,739.0,...,2969.0,10774.0,24183.0,9454.0,37243.0,30334.0,45669.0,40175.0,21531.0,8630.0
__,31623.0,150539.0,56012.0,75440.0,40270.0,22972.0,24039.0,19824.0,16094.0,9739.0,...,7244.0,70924.0,154396.0,60904.0,245476.0,200663.0,299340.0,243745.0,148416.0,58941.0
uncultured bacterium,14507.0,67151.0,24903.0,32921.0,21461.0,3978.0,4342.0,3516.0,2310.0,4689.0,...,362.0,2808.0,5635.0,2333.0,8973.0,7421.0,11071.0,9131.0,5250.0,1975.0


['uncultured bacterium', '__', 'uncultured bacterium', '__', 'uncultured bacterium', '__', '__', '__', '__', '__', 'uncultured bacterium', '__', '__', '__', 'Streptococcus mutans', 'uncultured bacterium', '__', '__', '__', '__', 'Rhodobacter sphaeroides', '__', '__', '__', '__', 'uncultured bacterium', '__', 'uncultured bacterium', '__', '__', '__', 'Acinetobacter baumannii', '__', 'Pseudomonas aeruginosa', '__', '__', '__', '__']


index,even_or_12_V8,even_or_13_V8,even_or_19_V8,even_or_20_V8,even_or_21_V8,even_out_001_V8,even_out_006_V8,even_out_007_V8,even_out_008_V8,even_out_009_V8,...,stag_out_011_V8,stag_st_01_V8,stag_st_02_V8,stag_st_03_V8,stag_st_04_V8,stag_st_05_V8,stag_st_06_V8,stag_st_07_V8,stag_st_08_V8,stag_st_09_V8
Acinetobacter baumannii,4316.0,19305.0,6226.0,7513.0,5670.0,127.0,3514.0,3460.0,3710.0,3411.0,...,103.0,153.0,218.0,105.0,378.0,328.0,426.0,411.0,240.0,133.0
Pseudomonas aeruginosa,2582.0,11230.0,3259.0,3914.0,2809.0,1966.0,3777.0,3602.0,2746.0,2116.0,...,752.0,758.0,1104.0,587.0,1995.0,1524.0,1962.0,1841.0,1330.0,612.0
Rhodobacter sphaeroides,3923.0,18053.0,5984.0,7056.0,5153.0,32747.0,4276.0,4619.0,4431.0,1895.0,...,6000.0,9862.0,0.0,7459.0,0.0,23294.0,28361.0,27834.0,17775.0,8504.0
Streptococcus mutans,3101.0,12942.0,4277.0,5221.0,3853.0,15630.0,710.0,663.0,4066.0,2154.0,...,10021.0,8123.0,0.0,6410.0,0.0,19233.0,24835.0,23174.0,15236.0,7088.0
__,16058.0,68706.0,22207.0,27562.0,18594.0,7658.0,12837.0,15745.0,11958.0,11894.0,...,8778.0,32773.0,74943.0,25972.0,136164.0,75790.0,93156.0,91915.0,56766.0,25320.0
uncultured bacterium,3352.0,15593.0,4413.0,5995.0,3693.0,1348.0,675.0,680.0,1249.0,899.0,...,1197.0,5662.0,3049.0,4394.0,5152.0,12993.0,16963.0,15315.0,8673.0,4040.0


['__', '__', '__', '__', '__', 'Staphylococcus aureus', '__', '__', 'Streptococcus mutans', '__', '__', '__', '__', '__', '__', '__', '__']


index,even_or_12_V9,even_or_13_V9,even_or_19_V9,even_or_20_V9,even_or_21_V9,even_out_006_V9,even_out_007_V9,even_out_008_V9,even_out_009_V9,even_out_012_V9,...,stag_out_011_V9,stag_st_01_V9,stag_st_02_V9,stag_st_03_V9,stag_st_04_V9,stag_st_05_V9,stag_st_06_V9,stag_st_07_V9,stag_st_08_V9,stag_st_09_V9
Staphylococcus aureus,0.0,17.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Streptococcus mutans,0.0,19.0,0.0,22.0,34.0,0.0,0.0,6.0,0.0,0.0,...,12.0,44.0,65.0,30.0,95.0,104.0,107.0,87.0,66.0,24.0
__,15886.0,74009.0,23769.0,30873.0,16998.0,16631.0,26107.0,28100.0,9872.0,2251.0,...,5067.0,30288.0,53579.0,24844.0,106430.0,78562.0,98581.0,96121.0,55756.0,23320.0
